In [1]:
# import the dependencies
import pandas as pd
import numpy as np 
import random
import matplotlib.pyplot as plt
from citipy import citipy

# import the datetime module from the datetime library
from datetime import datetime

import requests
from config import weather_api_key

In [2]:
# Creating a set of random latitude and Londitude combinations
lats = np.random.uniform(low= -90.000, high=90.000, size=2000)
lngs = np.random.uniform(low= -180.000, high=180.000, size=2000)
lats_lngs = zip(lats,lngs)
lats_lngs

In [3]:
# unpacking the zip and adding it to a list
coordinates = list(lats_lngs)

In [4]:
type(coordinates)

list

In [5]:
# starting URL for weather Map API call 
url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + weather_api_key

# Create a list for holing the cities:
cities =[]

# identify the nearest city for each latitude and longitude combination. 
for coordinate in coordinates:
    city = citipy.nearest_city( coordinate[0], coordinate[1]).city_name
    
    # if the city is unique, the we will add it to the list cities
    if city not in cities:
        cities.append(city)
        
# print the city count to confirm sufficient count.
len(cities)

774

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

Beginning Data Retrieval     
-----------------------------


In [10]:
# Create an empty list to hold the weather data.
city_data = []
# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
  
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0].get('description')    
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                            "Country": city_country,
                            "Lat": city_lat,
                            "Lng": city_lng,
                            "Max Temp": city_max_temp,
                            "Humidity": city_humidity,
                            "Cloudiness": city_clouds,
                            "Wind Speed": city_wind,
                            "Description": city_description,                          
                            "Date": city_date})
        # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        #pass

Processing Record 1 of Set 1 | bathsheba
Processing Record 2 of Set 1 | carnarvon
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | atuona
Processing Record 6 of Set 1 | nalut
Processing Record 7 of Set 1 | cherskiy
Processing Record 8 of Set 1 | mount gambier
Processing Record 9 of Set 1 | palabuhanratu
City not found. Skipping...
Processing Record 10 of Set 1 | buta
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | katangli
Processing Record 13 of Set 1 | lodja
Processing Record 14 of Set 1 | cidreira
Processing Record 15 of Set 1 | arraial do cabo
Processing Record 16 of Set 1 | gweru
Processing Record 17 of Set 1 | rio gallegos
Processing Record 18 of Set 1 | paamiut
Processing Record 19 of Set 1 | rikitea
Processing Record 20 of Set 1 | albany
Processing Record 21 of Set 1 | aracuai
Processing Record 22 of Set 1 | saskylakh
Processing Record 23 of Set 1 | klyuchevskiy
Processing Record 24 of Set 1 |

In [11]:
len(city_data)

710

In [12]:
# Convert the array of dictionaries to a pandas dictionary
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,Bathsheba,BB,13.22,-59.52,78.80,88,90,21.92,thunderstorm with light rain,2020-10-01 03:32:55
1,Carnarvon,AU,-24.87,113.63,71.60,68,0,23.04,clear sky,2020-10-01 03:32:46
2,Busselton,AU,-33.65,115.33,61.00,78,97,11.01,overcast clouds,2020-10-01 03:32:37
3,Ushuaia,AR,-54.80,-68.30,28.40,92,90,27.51,heavy snow,2020-10-01 03:32:30
4,Atuona,PF,-9.80,-139.03,78.49,76,3,20.89,clear sky,2020-10-01 03:32:16


In [19]:
# Reordering the columns, assigning a variable
new_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Description']
city_data_df= city_data_df[new_column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Bathsheba,BB,13.22,-59.52,78.80,88,90,21.92,thunderstorm with light rain
1,Carnarvon,AU,-24.87,113.63,71.60,68,0,23.04,clear sky
2,Busselton,AU,-33.65,115.33,61.00,78,97,11.01,overcast clouds
3,Ushuaia,AR,-54.80,-68.30,28.40,92,90,27.51,heavy snow
4,Atuona,PF,-9.80,-139.03,78.49,76,3,20.89,clear sky


In [21]:
# Create the output file (CSV)
output_data_file = 'WeatherPy_Database.csv'

# Export the contents of city_data into a CSV
city_data_df.to_csv(output_data_file, index_label='city_ID')